In [1]:
%run -i run_MDTVSFA.py
import os
home_dir = disk_dir_R + "tonemap/"
os.system('mkdir ' +home_dir)
os.system('mkdir ' +home_dir +  'VMAF_METRIX')
os.system('mkdir ' +home_dir +  'VMAF_METRIX/csv_res')
os.system('mkdir ' +home_dir +  'VMAF_METRIX/vid')

dst_dir = "P://7videos/" 
env = calc_met( model = "MDTVSFA", home_dir1=home_dir,dataset_dir=dst_dir)
all_method_name = ["gamma_unsharp", "CLAHE", "Tonemap_Drago", "Identity"]
all_func = [aug4const, expose_cv2, tonemapDrago, Identity]
args_d_Identity = args_d_CLAHE
all_argd = [args_d_aug4const, args_d_CLAHE, args_tonemapDrago, args_d_Identity]
SSIM_m = env.SSIM_metrix_get
PSNR_m = env.PSNR_metrix_get
all_proxy_metr = [SSIM_m, PSNR_m]
all_proxy_names = ["SSIM", "PSNR"]
exec(open('Current_model_lib.py').read())
met_name = "Linearity"
target_lst = [met_name]
k_lst = [1]
loss_calc = Custom_enh_Loss(target_lst = target_lst, k_lst=k_lst, to_train = False)
loss_calc = loss_calc.eval()
met_names = ["MDTVSFA", "Linearity", "SPAQ", "VSFA", "PAC2PIQ", "NIMA", "KONIQ"]
met_name = None

In [13]:
def get_met(X):
    global tmp
    tmp = X
    if met_name == "VSFA":
        return -loss_calc({"x_hat": torch.cat([X,X])}, torch.cat([X,X]))["loss"]
    else:
        return -loss_calc({"x_hat": X}, X)["loss"]
    
for met_name in met_names:
    print(met_name)
    target_lst = [met_name]
    k_lst = [1]
    loss_calc = Custom_enh_Loss(target_lst = target_lst, k_lst=k_lst, to_train = False)
    loss_calc.eval()
    loss_calc.get_met = get_met
    AV_log = []
    AV_av = []
    random.seed(a = 431)
    env.metr = get_met#
    toolbox = base.Toolbox()
    creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)
    toolbox.register("individual", init_range, creator.Individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evalOneMax)
    toolbox.register("mate", cxTwoPointCopy)
    toolbox.register("mutate", mut_cutom, indpb=1.)
    toolbox.register("select", tools.selNSGA2)
    creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

    for idx_proxy, idx_f  in tqdm(list(itertools.product(range(len(all_proxy_metr)), range(len(all_func))))):
        cur_proxy = all_proxy_metr[idx_proxy]
        cur_proxy_name = all_proxy_names[idx_proxy]
        cur_method_name, cur_func, cur_argd = list(zip(all_method_name, all_func, all_argd))[idx_f] 
        args_d = cur_argd
        print(cur_method_name + ", no codec tuning for "+ met_name + "_" + cur_proxy_name +", NSGA2")
        env.func = axis_swaper(cur_func)
        env.PSNR_metrix_get = cur_proxy
        stats = None
        AV_log = []
        AV_av = []
        for vid in tqdm(sorted(os.listdir(dst_dir))):
            pop = toolbox.population(n=28)
            env.dataset = [vid]
            env.init_video()
            if cur_method_name == "Identity":
                ngen = 1
                mu = 1
                lambda_ = 1
            else:
                ngen = 11
                mu = 14
                lambda_ = 17
            pop, logbook,statsmy = My_eaMuPlusLambda(pop, toolbox ,mu = mu,lambda_ = lambda_, cxpb=0.5, mutpb=0.49, ngen=ngen, stats=stats)
            AV_log.append(logbook)
            #with open(disk_dir + 'VMAF_METRIX/NeuralNetworkCompression/logs_black_box/' + met_name + "_" + cur_proxy_name + "_" + cur_method_name + ".npy" , 'wb') as f:
                #np.save(f, np.array(AV_log))

MDTVSFA


  0%|          | 0/8 [00:00<?, ?it/s]

gamma_unsharp, no codec tuning for MDTVSFA_SSIM, NSGA2


  0%|          | 0/7 [00:00<?, ?it/s]

E:\VMAF_METRIX\NeuralNetworkCompression\run_MDTVSFA.py:191: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  bst_ind = np.array(ind),ind.fitness.values[0], np.array([[[*i], [*i.fitness.values]] for i in pop ]), ind_value


...

  0%|          | 0/7 [00:00<?, ?it/s]

GEN:1  -115.85810089111328 -22.870561770854458 0.9226208925247192
GEN:2  -115.85810089111328 -24.749313781338355 0.9226208925247192
GEN:3  -118.24358367919922 -27.06193786282693 1.1089674234390259
GEN:4  -118.24358367919922 -25.866252510778367 1.1089674234390259
GEN:5  -118.24358367919922 -29.336327318222292 1.1089674234390259
GEN:6  -118.24358367919922 -28.660376577608048 1.1089674234390259
GEN:7  -118.24358367919922 -28.692895439363294 1.1089674234390259
GEN:8  -118.24358367919922 -27.801919179577983 1.1089674234390259
GEN:9  -118.24358367919922 -26.0400984479535 1.1089674234390259
GEN:10  -116.10581970214844 -26.67125512323072 1.1089674234390259
GEN:11  -116.10581970214844 -31.11250817006634 1.1089674234390259
BEST IND: (array([0.48823096, 2.99499898, 0.47982826]), 1.1089674234390259, array([[list([0.4882309596333681, 2.9949989843677325, 0.4798282595328519]),
        list([1.1089674234390259, 7.528516880169232])],
       [list([0.811340322862269, 1.4914493037691754, 0.89621824615754

GEN:1  -15.39091968536377 -1.9551124813095215 0.647197425365448
GEN:2  -10.485982894897461 -1.6595792693476523 0.647197425365448
GEN:3  -10.485982894897461 -2.0715461440624727 0.647197425365448
GEN:4  -9.961576461791992 -2.13908948052314 0.647197425365448
GEN:5  -9.961576461791992 -2.1473540606037265 0.647197425365448
GEN:6  -9.961576461791992 -1.8741413297191742 0.6473051309585571
GEN:7  -9.961576461791992 -1.9644998984952127 0.6473051309585571
GEN:8  -7.974786281585693 -1.3669419480908302 0.6473051309585571
GEN:9  -7.974786281585693 -1.479272554538423 0.6473051309585571
GEN:10  -7.974786281585693 -1.4934355078204986 0.6473051309585571
GEN:11  -7.974786281585693 -1.517040902568448 0.6473051309585571
BEST IND: (array([0.04841197, 2.4542029 , 0.25954582]), 0.6473051309585571, array([[list([0.04841197101462358, 2.4542028956782223, 0.25954581735852733]),
        list([0.6473051309585571, 12.583261152114938])],
       [list([1.031828600829685, 1.350096396934335, 1.0]),
        list([-7.974

  0%|          | 0/7 [00:00<?, ?it/s]

GEN:1  -113.16462707519531 -56.582313537597656 0
BEST IND: [0, 0]
GEN:1  -99.04391479492188 -49.52195739746094 0
BEST IND: [0, 0]
GEN:1  -72.47567749023438 -36.23783874511719 0
BEST IND: [0, 0]
GEN:1  -13.579489707946777 -6.789744853973389 0
BEST IND: [0, 0]
GEN:1  -11.507516860961914 -5.753758430480957 0
BEST IND: [0, 0]
GEN:1  -46.97008514404297 -23.485042572021484 0
BEST IND: [0, 0]
GEN:1  9.013235092163086 4.506617546081543 9.013235092163086
BEST IND: (array([35.39765793, 15.59674838]), 9.013235092163086, array([[[35.39765793, 15.59674838],
        [ 9.01323509, 60.        ]]]), 9.013235092163086)
